<a href="https://colab.research.google.com/github/Anilesh05/Anilesh/blob/main/Find_tags_for_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Downloading and installing hadoop***

In [ ]:
!apt-get install openjdk-8-jdk
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar fx hadoop-3.3.6.tar.gz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
!ln -s /content/hadoop-3.3.6/bin/* /usr/bin

# ***Create ' movie_lens_data.txt ' file***

In [ ]:
%%writefile movie_lens_data.txt

UserID::MovieID::Moviename::Tag::Timestamp
01::01::Spider-Man: No Way Home (2021)::action::1648847400
01::01::Spider-Man: No Way Home (2021)::adventure::1648847400
02::02::Dune (2021)::sci-fi::1648847400
02::02::Dune (2021)::fantasy::1648847400
03::03::The Matrix Resurrections (2021)::action::1648847400
03::03::The Matrix Resurrections (2021)::sci-fi::1648847400
04::04::Black Widow (2021)::action::1648847400
04::04::Black Widow (2021)::adventure::1648847400
05::05::Shang-Chi and the Legend of the Ten Rings (2021)::action::1648847400
05::05::Shang-Chi and the Legend of the Ten Rings (2021)::fantasy::1648847400
06::06::No Time to Die (2021)::action::1648847400
06::06::No Time to Die (2021)::adventure::1648847400
07::07::Eternals (2021)::action::1648847400
07::07::Eternals (2021)::fantasy::1648847400
08::08::Free Guy (2021)::comedy::1648847400
08::08::Free Guy (2021)::adventure::1648847400
09::09::Jungle Cruise (2021)::adventure::1648847400
09::09::Jungle Cruise (2021)::fantasy::1648847400
10::10::Venom: Let There Be Carnage (2021)::action::1648847400
10::10::Venom: Let There Be Carnage (2021)::sci-fi::1648847400
11::11::Inception (2010)::sci-fi::1648847400
11::11::Inception (2010)::thriller::1648847400
12::12::The Dark Knight (2008)::action::1648847400
12::12::The Dark Knight (2008)::crime::1648847400
13::13::Interstellar (2014)::sci-fi::1648847400
13::13::Interstellar (2014)::adventure::1648847400
14::14::Fight Club (1999)::drama::1648847400
14::14::Fight Club (1999)::psychological::1648847400
15::15::The Shawshank Redemption (1994)::drama::1648847400
15::15::The Shawshank Redemption (1994)::inspirational::1648847400
16::16::Pulp Fiction (1994)::crime::1648847400
16::16::Pulp Fiction (1994)::black comedy::1648847400
17::17::Forrest Gump (1994)::drama::1648847400
17::17::Forrest Gump (1994)::romance::1648847400
18::18::The Godfather (1972)::crime::1648847400
18::18::The Godfather (1972)::mafia::1648847400
19::19::The Lord of the Rings: The Fellowship of the Ring (2001)::fantasy::1648847400
19::19::The Lord of the Rings: The Fellowship of the Ring (2001)::adventure::1648847400
20::20::The Matrix (1999)::action::1648847400
20::20::The Matrix (1999)::cyberpunk::1648847400


In [ ]:
%%writefile mapper.py
#!/usr/bin/env python

import sys

for line in sys.stdin:
    fields = line.strip().split("::")
    if len(fields) == 5:
        movie_id, movie_name, tags = fields[1], fields[2], fields[3].split(",")
        for tag in tags:
            print('%s\t%s\t%s' % (movie_id, movie_name, tag))

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

# Initialize variables
rows = {}

# Print column names
print('{:<10} {:<60} {:<20}'.format("MovieID", "Moviename", "Tags"))
print()

# Input comes from STDIN
for line in sys.stdin:
    # Split the line into movie_id, movie_name, and tag
    movie_id, movie_name, tag = line.strip().split('\t')

    # Add movie_name and tag to the rows dictionary
    if movie_id in rows:
        rows[movie_id][1] += f", {tag}"  # Concatenate tags
    else:
        rows[movie_id] = [movie_name, tag]

# Output the table format

for movie_id, (movie_name, tags) in rows.items():
    print('{:<10} {:<60} {:<20}'.format(movie_id, movie_name, tags))


In [ ]:
!hdfs dfs -mkdir input

In [ ]:
!hdfs dfs -mv movie_lens_data.txt input/

In [ ]:
!hdfs dfs -cat input/movie_lens_data.txt

In [ ]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -files mapper.py,reducer.py \
    -mapper mapper.py \
    -reducer reducer.py \
    -input input \
    -output output

In [ ]:
!cat output/part-00000